### Libraries

In [1]:
import pandas as pd
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### API Keys

In [7]:
os.environ["OPENAI_API_KEY"] = "hidden"
os.environ["GOOGLE_API_KEY"] = "hidden"

In [5]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

### Code generation prompts

In [8]:
system_prompt = """
You are a helpful assistant, assinged with generating code to python functions and classes when given a docstring. 
You have vast knowledge of python coding and libraries.
You know and follow the google docstring conventions.
"""

my_prompt = """
Provide a python code to the following docstring: {docstring}

Correct format example:

docstring = 

Iteratively compute the sum of all elements in a list of integers after multiplying each element by its index in the list.
    
Args:
lst (list): a list of integers.

Returns:
int: the sum of all elements in the list after preforming the multipication of the elements by their indices.

solution =
    
def multiply_and_sum(lst):
    res = 0
    for i, item in enumerate(lst):
        res += item * i 
    return res
        
Return the code only, do not include the docstring in your answer. 
"""

### LLM Setup

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_prompt),
])

In [10]:
def generate_code_from_docstring(docstring, chain_solution):
    llm_code = chain_solution.invoke({
        "docstring": docstring
    })
    return llm_code

### OPEN AI

In [11]:
llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

output_parser = StrOutputParser()

chain_gpt_turbo = prompt | llm_openai | output_parser

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


First, generate code for one function and class based on gemini docstring

In [13]:
data = pd.read_csv("data_full_docstrings_generated.csv")
function_2_docstring_gemini = data.loc[2, 'Gemini-1.0-pro']
class_4_docstring_gemini = data.loc[43, 'Gemini-1.0-pro']

In [16]:
generated_code_function_2_gpt_turbo = generate_code_from_docstring(function_2_docstring_gemini, chain_gpt_turbo)
generated_code_class_4_gpt_turbo = generate_code_from_docstring(class_4_docstring_gemini, chain_gpt_turbo)

print(f"Function 2 generated code: \n{generated_code_function_2_gpt_turbo}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gpt_turbo}")

Function 2 generated code: 
def recursive_sum_nested_list(lst):
    if not lst:
        return 0
    total = 0
    for element in lst:
        if isinstance(element, list):
            total += recursive_sum_nested_list(element)
        else:
            total += element
    return total

Class 4 generated code: 
```python
class Point:
    """
    A class used to represent a 2D point.

    Attributes:
    ----------
    x : float 
        the x-coordinate of the point.
    y : float 
        the y-coordinate of the point.
    r : float 
        the polar coordinate of the point.
    theta : float 
        the polar coordinate of the point.

    Methods:
    -------
    __init__(x, y)
        initializes the Point with x and y coordinates.
    __repr__()
        returns a string representation of the point.
    __eq__(other)
        compares the point to another point.
    __add__(other)
        adds two points together.
    __sub__(other)
        subtracts two points.
    distance(othe

### GOOGLE

In [17]:
prompt_for_gemini = ChatPromptTemplate.from_template(
    system_prompt + my_prompt
)

llm_google = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.0-pro")

chain_gemini = prompt_for_gemini | llm_google | output_parser

In [18]:
generated_code_function_2_gemini = generate_code_from_docstring(function_2_docstring_gemini, chain_gemini)
generated_code_class_4_gemini = generate_code_from_docstring(class_4_docstring_gemini, chain_gemini)

print(f"Function 2 generated code: \n{generated_code_function_2_gemini}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gemini}")

Function 2 generated code: 
def sum_of_nested_list(lst):
    total = 0
    for item in lst:
        if isinstance(item, list):
            total += sum_of_nested_list(item)
        else:
            total += item
    return total

Class 4 generated code: 
```python
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = (x**2 + y**2)**0.5
        self.theta = np.arctan2(y, x)

    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)

    def distance(self, other):
        return ((self.x - other.x)**2 + (self.y - other.y)**2)**0.5

    def angle_wrt_origin(self, other):
        return np.arccos((self.x * other.x + self.y * other.y) / (self.r * other.r))
```


Generate code using GPT 4

In [19]:
llm_openai_gpt4 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-1106-preview")

output_parser = StrOutputParser()

chain_gpt4 = prompt | llm_openai_gpt4 | output_parser

In [20]:
generated_code_function_2_gpt4 = generate_code_from_docstring(function_2_docstring_gemini, chain_gpt4)
generated_code_class_4_gpt4 = generate_code_from_docstring(class_4_docstring_gemini, chain_gpt4)

print(f"Function 2 generated code: \n{generated_code_function_2_gpt4}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gpt4}")

Function 2 generated code: 
```python
def recursive_sum(lst):
    total = 0
    for element in lst:
        if isinstance(element, list):
            total += recursive_sum(element)
        else:
            total += element
    return total
```

Class 4 generated code: 
```python
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = math.sqrt(x**2 + y**2)
        self.theta = math.atan2(y, x)

    def __repr__(self):
        return f'Point({self.x}, {self.y})'

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)

    def distance(self, other):
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

    def angle_wrt_origin(self):
        return math.atan2(self.y, self.x)
```


Generate code for all models and docstrings

In [23]:
generated_docstrings_data = pd.read_csv("data_full_docstrings_generated.csv")

models = ['T5', 'Gemini-1.0-pro', 'GPT-3.5 Turbo', 'Claude-instant-1']

tqdm.pandas()

for model_name in models:
    new_code_col = f'generated code for {model_name} docstring'
    generated_docstrings_data[new_code_col] = None
    missing_values_mask = generated_docstrings_data[new_code_col].isna()
    generated_docstrings_data.loc[missing_values_mask, [new_code_col]] = (
        generated_docstrings_data.loc[missing_values_mask, model_name].progress_apply(
            lambda x: generate_code_from_docstring(x, chain_gpt4)
        )
    )
    # Debugging: Print the generated docstrings
    print(generated_docstrings_data.loc[missing_values_mask, [new_code_col]])

generated_docstrings_data.to_csv("code_gen_based_on_docstrings_GPT4.csv", index=False)

  0%|          | 0/50 [00:00<?, ?it/s]

                      generated code for T5 docstring
0   ```python\ndef sum_delimited_items(lst, delimi...
1   ```python\ndef multiply_and_sum(lst):\n    res...
2   ```python\ndef multiply_and_sum(lst):\n    res...
3   ```python\ndef count_words_in_string(input_str...
4   ```python\ndef count_k_columns_greater_than_va...
5   ```python\ndef get_nice_tree_index():\n    # I...
6   ```python\ndef multiply_and_sum(lst):\n    res...
7   ```python\nimport random\n\ndef distance_betwe...
8   ```python\ndef is_dag(graph):\n    visited = s...
9   ```python\ndef multiply_and_sum(lst):\n    res...
10  ```python\ndef diff_matrices(matrix_list):\n  ...
11  ```python\ndef longest_subsequence_length(seq)...
12  ```python\ndef find_media_item(store, minimum)...
13  ```python\ndef find_primary_factors(number):\n...
14  ```python\ndef intersection_of_graphs(graph1, ...
15  ```python\ndef sum_combinations(total_sum):\n ...
16  ```python\ndef multiply_and_sum(lst):\n    res...
17  ```python\ndef is_valid_

  0%|          | 0/50 [00:00<?, ?it/s]

          generated code for Gemini-1.0-pro docstring
0   ```python\ndef sum_even_indexed_elements(lst):...
1   ```python\ndef count_ways_to_make_change(n, ls...
2   ```python\ndef sum_nested_list(lst):\n    tota...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef binomial_coefficient(n, k):\n  ...
5   ```python\ndef dfs_level_order(tree, index=0):...
6   ```python\ndef find_subset_with_half_sum(lst):...
7   ```python\ndef edit_distance(x, y):\n    m, n ...
8   ```python\ndef is_dag(graph):\n    def has_cyc...
9   ```python\ndef compute_sequence_value(num, x=0...
10  ```python\ndef subtract_sparse_matrices(lst):\...
11  ```python\ndef longest_subsequence_length(lst)...
12  ```python\ndef quickselect(nums, k):\n    """\...
13  ```python\ndef prime_factors(n):\n    factors ...
14  ```python\ndef intersect_graphs(g1, g2):\n    ...
15  ```python\ndef subset_sum(lst, target):\n    d...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef has_repea

  0%|          | 0/50 [00:00<?, ?it/s]

           generated code for GPT-3.5 Turbo docstring
0   ```python\ndef sum_even_elements(lst):\n    de...
1   ```python\ndef count_ways_to_sum(n, lst):\n   ...
2   ```python\ndef recursive_sum_abs_floats(lst):\...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef choose(n, k):\n    if k == 0 or...
5   ```python\ndef depth_first_search(tree, index=...
6   ```python\ndef find_subset_with_half_sum(lst):...
7   ```python\ndef min_string_distance(x, y):\n   ...
8   def is_dag(graph):\n    def visit(node):\n    ...
9   ```python\ndef calculate_values(num, x):\n    ...
10  def subtract_sparse_matrices(lst):\n    if not...
11  ```python\ndef longest_subsequence(lst):\n    ...
12  ```python\ndef quickselect(nums, k):\n    if l...
13  ```python\ndef find_primary_factors(n):\n    f...
14  ```python\ndef find_intersection_of_graphs(g1,...
15  ```python\ndef find_subsets_with_target_sum(ls...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef has_repea

  0%|          | 0/50 [00:00<?, ?it/s]

        generated code for Claude-instant-1 docstring
0   ```python\ndef sum_even_indexed_values(lst):\n...
1   ```python\ndef change_possibilities(n, lst):\n...
2   ```python\ndef absolute_sum_of_nested(lst):\n ...
3   ```python\ndef decompose_count(target, word_ba...
4   def compute_nCk(n, k):\n    def factorial(num)...
5   ```python\nfrom collections import deque\n\nde...
6   ```python\ndef find_subset_half_sum(lst):\n   ...
7   ```python\ndef levenshtein_distance(x, y):\n  ...
8   ```python\ndef is_dag(graph):\n    def dfs(nod...
9   ```python\ndef foo(num, x=0):\n    if num < 10...
10  ```python\ndef subtract_sparse_matrices(matric...
11  ```python\ndef find_longest_subsequences(lst):...
12  ```python\ndef quickselect(nums, k):\n    if l...
13  ```python\ndef prime_factors(n):\n    factors ...
14  ```python\ndef intersect_graphs(g1, g2):\n    ...
15  ```python\ndef find_subsets(lst, target):\n   ...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef has_seen_

### gemini 1.5 flash - free

In [21]:
llm_google_flash = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.5-flash")

chain_gemini_flash = prompt_for_gemini | llm_google_flash | output_parser

In [22]:
generated_code_function_2_gemini = generate_code_from_docstring(function_2_docstring_gemini, chain_gemini_flash)
generated_code_class_4_gemini = generate_code_from_docstring(class_4_docstring_gemini, chain_gemini_flash)

print(f"Function 2 generated code: \n{generated_code_function_2_gemini}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gemini}")

Function 2 generated code: 
```python
def sum_nested_list(lst):
    total = 0
    for element in lst:
        if isinstance(element, list):
            total += sum_nested_list(element)
        else:
            total += element
    return total
```

Class 4 generated code: 
```python
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = math.sqrt(x**2 + y**2)
        self.theta = math.atan2(y, x)

    def __repr__(self):
        return f"Point({self.x:.2f}, {self.y:.2f})"

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        return NotImplemented

    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.x - other.x, self.y - other.y)
        return NotImplemented


In [41]:
generated_docstrings_data = pd.read_csv("data_full_docstrings_generated.csv")
generated_docstrings_data.rename({'T5 BaseLine docstring generation': 'T5'}, axis=1, inplace=True)  # column name is too long

models = ['T5', 'Gemini-1.0-pro', 'GPT-3.5 Turbo', 'Claude-instant-1']

tqdm.pandas()

for model_name in models:
    new_code_col = f'generated code for {model_name} docstring'
    generated_docstrings_data[new_code_col] = None
    missing_values_mask = generated_docstrings_data[new_code_col].isna()
    generated_docstrings_data.loc[missing_values_mask, [new_code_col]] = (
        generated_docstrings_data.loc[missing_values_mask, model_name].progress_apply(
            lambda x: generate_code_from_docstring(x, chain_gemini_flash)
        )
    )
    # Debugging: Print the generated docstrings
    print(generated_docstrings_data.loc[missing_values_mask, [new_code_col]])

generated_docstrings_data.to_csv("code_gen_based_on_docstrings_gemini_flash.csv", index=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

                      generated code for T5 docstring
0   ```python\ndef sum_delimited(lst, delimiter):\...
1   ```python\ndef find_the_number_of_changes(s1, ...
2   ```python\ndef sum_nested_structures(structure...
3   ```python\ndef count_words(text):\n  """Gets t...
4   ```python\ndef count_k_columns(matrix, k):\n  ...
5   ```python\ndef get_index_in_tree(tree, target)...
6   ```python\ndef get_half_n_sum_set(n_sum_set, n...
7   ```python\ndef partition_sets_to_subsets(parti...
8   ```python\ndef is_dag(graph):\n    """Determin...
9   ```python\nimport random\nimport datetime\n\nd...
10  ```python\ndef diff_matrices(matrices):\n  """...
11  ```python\ndef longest_subsequence(s1, s2):\n ...
12  ```python\ndef count_mobile_characters(table):...
13  ```python\ndef find_prime_factors(num):\n    "...
14  ```python\ndef find_intersection(graph1, graph...
15  ```python\ndef calculate_subset_sum(target, su...
16  ```python\ndef sum_mult_str(evals):\n  sum_mul...
17  ```python\ndef is_valid_

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<l

...

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

           generated code for GPT-3.5 Turbo docstring
0   ```python\ndef sum_even_indexed_elements(lst):...
1   ```python\ndef count_representations(n, lst):\...
2   ```python\ndef sum_nested_list(lst):\n    tota...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef combinations(n, k):\n  """Calcu...
5   ```python\ndef dfs_level_order(tree, index=0):...
6   ```python\ndef find_subset_with_half_sum(lst):...
7   ```python\ndef stirling_number_second_kind(n, ...
8   ```python\ndef is_dag(graph):\n    def dfs(nod...
9   ```python\ndef recursive_calc(num, x=0):\n    ...
10  ```python\ndef sparse_matrix_difference(lst):\...
11  ```python\ndef longest_increasing_or_decreasin...
12  ```python\ndef median(nums):\n    nums.sort()\...
13  ```python\ndef find_prime_factors(n):\n    fac...
14  ```python\ndef graph_intersection(g1, g2):\n  ...
15  ```python\ndef find_subsets_with_sum(lst, targ...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef check_rep

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

        generated code for Claude-instant-1 docstring
0   ```python\ndef sum_even(lst):\n    total = 0\n...
1   ```python\ndef count_change(n, lst):\n    if n...
2   ```python\ndef nested_sum(lst):\n    total = 0...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef binomial_coefficient(n, k):\n  ...
5   ```python\ndef dfs_level_order(tree, index=0):...
6   ```python\ndef find_half_sum_subset(lst):\n   ...
7   ```python\ndef count_partitions(n, k):\n    me...
8   ```python\ndef is_dag(graph):\n    def dfs(nod...
9   ```python\ndef recursive_function(num, x=0):\n...
10  ```python\ndef sparse_matrix_difference(lst):\...
11  ```python\ndef longest_subsequence(lst):\n    ...
12  ```python\ndef median(nums):\n    nums = list(...
13  ```python\ndef find_prime_factors(n):\n    fac...
14  ```python\ndef intersection_graph(g1, g2):\n  ...
15  ```python\ndef generate_subsets(lst, target):\...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef check_sub